In [11]:
import jieba
import json
import re
import os
import re
import chardet

In [3]:
# 文件批处理
def FileReader(tag):
    # print(os.getcwd())
    path = r'../ex_5/data/htl_del_4000/' + tag 
    os.listdir(path)
    FileList = []
    for iter in os.listdir(path):
        if os.path.splitext(iter)[1] == '.txt':
            FileList.append(iter)
    return FileList


In [ ]:
# 对语料进行分词，去除停用词

def _PreProcess(article):
    article = re.sub('\\s','',article)
    article = re.sub('[a-z,A-Z]*','',article)
    sentence = article.split('。|？|！')
    row_list =[]
    for iter in sentence:
        row_list.append(jieba.lcut(iter,cut_all = False)) 
    # print(type(row_list))

    return row_list

def PreProcess(path,id_f):
    text = ''
    # print(path)
    with open(path,'r',encoding = 'gbk',errors = 'ignore') as txt:
        text = txt.readline()
    # print(text[:20])
    sentences = _PreProcess(text)
    # print(type(sentences))
    dic = {}
    id_c = 0
    for iter in sentences:
        dic[id_c] = iter
        id_c += 1
    return dic



def File_Dic(FileList,tag):
    path = r'../ex_5/data/htl_del_4000/' + tag + r'/'
    dic = {}

    for iter in range(0,len(FileList)):
        cur_path = path + FileList[iter]
        dic[iter] = PreProcess(cur_path,iter)
        # print(dic[iter])
    return dic


def row_split(tag):
    FileList = FileReader(tag)
    # ReWrite(FileList,tag)
    volc = File_Dic(FileList,tag)
    # print(len(FileList))
    j_data = json.dumps(volc,ensure_ascii=False)
    with open(r'./row_语料分词_' + tag + '.json' ,'w',encoding='utf-8') as f:
        f.write(j_data)

row_split('pos')
row_split('neg')

In [12]:
# 加载分词后的实验数据，获取所有的句子，计算句子的向量表示
def load_sentence(tag):
    sentence = []
    data = {}
    with open(r'./row_语料分词_' + tag + '.json' ,'r',encoding='utf-8') as f:
        data = json.load(f)
        for iter in data.values():
            for sent in iter.values():
                sentence.append(sent)
    return sentence
sentencePos = load_sentence('pos')
sentenceNeg = load_sentence('neg')
# print(sentencePos[:10])


In [19]:
def _countF(set,sentences):
    dic = {}
    for iter in set:
        for sentence in sentences:
            if iter not in dic:
                dic[iter] = 0
            if iter in sentence:
                dic[iter] += 1
    return dic

# 测试
# c = ['a','e','i','o','u']
# c = set(c)
# word = ['my','supper','countless']
# dic = _countF(c,word)
# print(dic)

In [7]:
import numpy as np
def softmax(x):
    x = np.array(x)
    f_x = np.exp(x) / np.sum(np.exp(x))
    
    return list(f_x)

In [20]:
# 计算句子中每个词的 tf-idf
import math 

sentenceMerge = sentencePos + sentenceNeg
MergeSet = []
for iter in sentencePos:
    MergeSet += iter

for iter in sentenceNeg:
    MergeSet += iter

MergeSet = set(MergeSet)
# 计算TF-idf

freq = _countF(MergeSet,sentenceMerge)

Vector_TF_IDF = []
for sentence in sentenceMerge:
    tmp_TF_IDF = []
    for iter in MergeSet:
        if iter in sentence:
            tmp_TF_IDF.append(float(sentence.count(iter))/len(MergeSet) * math.log(float(len(sentenceMerge))/freq[iter]))
        else:
            tmp_TF_IDF.append(0)
    
    # 过soft max函数
    tmp_TF_IDF = softmax(tmp_TF_IDF)
    Vector_TF_IDF.append(tmp_TF_IDF)



In [23]:
print(type(list(Vector_TF_IDF[0])))

<class 'list'>


In [27]:
# 存储为 类别 + 向量
with open (r'./句子的向量表示' ,'w',encoding= 'utf-8') as f:
    for senctence in range (0,len(sentencePos)):
        f.write( '1\t' + str(Vector_TF_IDF[senctence]) + '\n')
    for senctence in range (len(sentencePos),len(sentenceMerge)):
        f.write( '0\t' + str(Vector_TF_IDF[senctence]) + '\n')
        


In [35]:
# 序列化存储
import pickle
with open (r'./句子的向量表示_序列化' ,'wb') as f:
    dic = {0:[],1:[]}
    for sentence in range (0,len(sentencePos)):
        dic[1].append(list(Vector_TF_IDF[sentence]))
    for sentence in range (len(sentencePos),len(sentenceMerge)):
        dic[0].append(list(Vector_TF_IDF[sentence]))
        # f.write( '0\t' + str(Vector_TF_IDF[senctence]) + '\n')
    pickle.dump(dic,f)